In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install autogluon

In [ ]:
# !pip install -U "mxnet_cu101<2.0.0"

In [ ]:
from autogluon.tabular import TabularDataset,TabularPredictor

In [ ]:
train_data_path = '/kaggle/input/california-house-prices/train.csv'
test_data_path = '/kaggle/input/california-house-prices/test.csv'

train_data = TabularDataset(train_data_path)

In [ ]:
id, label = 'Id','Sold Price'
large_val_cols = ['Lot','Total interior livable area','Tax assessed value','Annual tax amount','Listed Price','Last Sold Price']
for c in large_val_cols + [label]:
    train_data[c] = np.log(train_data[c]+1)

In [ ]:
# predictor = TabularPredictor(label=label).fit(train_data.drop(columns=[id]),time_limit=3600)
# predictor = TabularPredictor(label=label).fit(train_data.drop(columns=[id]),ag_args_fit={'num_gpus': 1})

predictor = TabularPredictor(label=label).fit(train_data.drop(columns=[id]),
                                             hyperparameters='multimodal', # 使用transformer抽取特征,使用多模型融合
                                             num_stack_levels=1,# ensemble 集成学习
                                             num_bag_folds=5,time_limit=3600)

In [ ]:
test_data = TabularDataset(test_data_path)
for c in large_val_cols:
    test_data[c] = np.log(test_data[c]+1)

In [ ]:
y_pred = predictor.predict(test_data.drop(columns=[id]))

In [ ]:
y_pred = np.exp(y_pred) - 1

In [ ]:
y_pred[:3]

In [ ]:
test_data[:3]

In [ ]:
save_path = '/kaggle/working/submit.csv'

submit_result = pd.DataFrame({'id': test_data[id],'Sold Price':y_pred})
submit_result

In [ ]:
submit_result.to_csv(save_path,index=False)

In [ ]:
nan

In [ ]:
# 更好的模型
# predictor = TabularPredictor(label=label).fit(train_data.drop(columns=[id]),
#                                              hyperparameters='multimodal', # 使用transformer抽取特征,使用多模型融合
#                                              num_stack_levels=1,# ensemble 集成学习
#                                              num_bag_folds=5)

In [ ]:
nan